# Загрузка всех моделей

## Эмбеддинги

In [43]:
from langchain_community.embeddings import InfinityEmbeddings

emb_model_BERTA = InfinityEmbeddings(model="sergeyzh/BERTA", infinity_api_url="http://127.0.0.1:7997")
emb_model_USER2 = InfinityEmbeddings(model="deepvk/USER2-base", infinity_api_url="http://127.0.0.1:7997")
emb_model_RU_EN = InfinityEmbeddings(model="ai-forever/ru-en-RoSBERTa ", infinity_api_url="http://127.0.0.1:7997")


## LLM

In [9]:
import re
import os
from typing import Optional, Union
from deepeval.models import DeepEvalBaseLLM
from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel
import instructor

class SGlangModel(DeepEvalBaseLLM):
    def __init__(self, 
                 model_name: str, 
                 base_url: str, 
                 api_key: Optional[str] = "NET",
                 enable_thinking: bool = False): # Параметр для управления режимом мышления Qwen3
        """
        Инициализирует модель SGlang.

        Args:
            model_name (str): Имя модели.
            base_url (str): Базовый URL для API.
            api_key (Optional[str]): API ключ. По умолчанию "NET".
            enable_thinking (bool): Флаг для управления поведением моделей Qwen3.
                                     Если True, к промпту для Qwen3 будет добавлен "/think" (для стимуляции процесса размышления).
                                     Если False, к промпту для Qwen3 будет добавлен "/no_think".
                                     Вне зависимости от этого флага, блок <think> будет удален из финального вывода
                                     для Qwen3 в режиме генерации текста (когда schema is None).
                                     По умолчанию False.
        """
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY")
        self.enable_thinking = enable_thinking 
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        if self._async_client is None:
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._async_client

    def _clean_qwen3_output(self, text_response: str) -> str:
        """
        Удаляет начальный блок <think>...</think> из ответов модели Qwen3.
        """
        pattern = r'^\s*<think>.*?</think>\s*'
        cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)
        return cleaned_response

    def generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:
        """
        Генерирует ответ от модели. 
        Для модели 'Qwen3' (без схемы):
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.
        - Блок <think> всегда удаляется из финального ответа.
        """
        client = self.load_model()
        
        processed_prompt = prompt
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None

        if is_qwen3_text_mode:
            # Удаляем существующие теги /think или /no_think из конца промпта
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            
            if self.enable_thinking:
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления
            else:
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления
        
        try:
            if schema is None:
                response = client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": processed_prompt}], 
                )
                raw_content = response.choices[0].message.content
                
                if is_qwen3_text_mode:
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>.
                    # self.enable_thinking контролирует, получает ли модель /think или /no_think в промпте,
                    # т.е. будет ли она проходить через процесс размышления.
                    return self._clean_qwen3_output(raw_content)
                else:
                    # Для других моделей возвращаем "сырой" контент
                    return raw_content
            else:
                # Режим структурированного вывода с instructor
                instructor_client = instructor.from_openai(
                    client=client,
                    mode=instructor.Mode.JSON 
                )
                response_obj = instructor_client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}], 
                    response_model=schema, 
                )
                return response_obj
        except Exception as e:
            print(f"Ошибка при синхронной генерации VLLM/Instructor для промпта '{prompt[:50]}...': {e}")
            raise e

    async def a_generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:
        """
        Асинхронно генерирует ответ от модели.
        Для модели 'Qwen3' (без схемы):
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.
        - Блок <think> всегда удаляется из финального ответа.
        """
        client = self.load_async_model()

        processed_prompt = prompt
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None

        if is_qwen3_text_mode:
            # Удаляем существующие теги /think или /no_think из конца промпта
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()

            if self.enable_thinking:
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления
            else:
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления

        try:
            if schema is None:
                response = await client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": processed_prompt}], 
                )
                raw_content = response.choices[0].message.content

                if is_qwen3_text_mode:
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>.
                    # self.enable_thinking контролирует, получает ли модель /think или /no_think в промпте.
                    return self._clean_qwen3_output(raw_content)
                else:
                    # Для других моделей возвращаем "сырой" контент
                    return raw_content
            else:
                # Режим структурированного вывода с instructor
                instructor_client = instructor.from_openai(
                    client=client,
                    mode=instructor.Mode.JSON
                )
                response_obj = await instructor_client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}], 
                    response_model=schema,
                )
                return response_obj
        except Exception as e:
            print(f"Ошибка при асинхронной генерации VLLM/Instructor для промпта '{prompt[:50]}...': {e}")
            raise e

    def get_model_name(self) -> str:
        return self.model_name

In [10]:
Qwen3_8_SGlang = SGlangModel(model_name="Qwen/Qwen3-8B", base_url="http://85.143.167.11:30000/v1")
Qwen3_8_SGlang_Reasoning = SGlangModel(model_name="Qwen/Qwen3-8B", base_url="http://85.143.167.11:30000/v1", enable_thinking = True)

In [11]:
from langchain_openai import ChatOpenAI

Cogito = ChatOpenAI(model="deepcogito/cogito-v1-preview-llama-8B", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Qwen3_8 = ChatOpenAI(model="Qwen/Qwen3-8B", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Gemma = ChatOpenAI(model="google/gemma-3-4b-it", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Yandex = ChatOpenAI(model="yandex/YandexGPT-5-Lite-8B-instruct", base_url = "http://85.143.167.11:30000/v1", api_key="NO")



---

## Чанкинг

In [12]:
file_path_1 = "/home/lanarich/Рабочий стол/Diploma/data/output_markdown/Anatomia_cheloveka_1_tom_2.md"
file_path_2 = "/home/lanarich/Рабочий стол/Diploma/data/output_markdown/Kapandzhi_-_Pozvonochnik.md" 
file_path_3 = "/home/lanarich/Рабочий стол/Diploma/data/output_markdown/Kozhnye_i_venericheskie_bolezni_pod_red_O_Yu_Olisovoi_774.md" 
file_path_4 = "/home/lanarich/Рабочий стол/Diploma/data/output_markdown/Molekulyarnaya_biologia_kletki_Tom_1.md" 

with open(file_path_1, 'r', encoding='utf-8') as file:
    book_1 = file.read()

with open(file_path_2, 'r', encoding='utf-8') as file:
    book_2 = file.read()

with open(file_path_3, 'r', encoding='utf-8') as file:
    book_3 = file.read()

with open(file_path_4, 'r', encoding='utf-8') as file:
    book_4 = file.read()

In [13]:
from chonkie import TokenChunker

import tiktoken
tokenizer = tiktoken.get_encoding("o200k_base")
chunker_token = TokenChunker(
    tokenizer=tokenizer,
    chunk_size=512,
    chunk_overlap=128,
    return_type= "texts"
)

In [14]:
book_1_token = chunker_token.chunk(book_1)
book_2_token = chunker_token.chunk(book_2)
book_3_token = chunker_token.chunk(book_3)
book_4_token = chunker_token.chunk(book_4)
full_books_token = book_1_token + book_2_token + book_3_token + book_4_token

In [15]:
from chonkie import RecursiveChunker

chunker_rec = RecursiveChunker.from_recipe(
        name="markdown",
        chunk_size=512,
        tokenizer_or_token_counter=tokenizer, 
        min_characters_per_chunk=75,         
        return_type='texts'          
)

In [16]:
book_1_rec = chunker_rec.chunk(book_1)
book_2_rec = chunker_rec.chunk(book_2)
book_3_rec = chunker_rec.chunk(book_3)
book_4_rec = chunker_rec.chunk(book_4)
full_books_rec = book_1_rec + book_2_rec + book_3_rec + book_4_rec

In [17]:
from chonkie import LateChunker
chunker_late = LateChunker.from_recipe(
        name = "markdown",
        embedding_model="sergeyzh/BERTA",
        chunk_size=512, 
        min_characters_per_chunk=75
)

Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


In [ ]:
book_1_late = chunker_late.chunk(book_1)
book_2_late = chunker_late.chunk(book_2)
book_3_late = chunker_late.chunk(book_3)
book_4_late = chunker_late.chunk(book_4)
full_books_late = book_1_late + book_2_late + book_3_late + book_4_late

Token indices sequence length is longer than the specified maximum sequence length for this model (665 > 512). Running this sequence through the model will result in indexing errors
/home/lanarich/.cache/pypoetry/virtualenvs/diploma-au9TFMuk-py3.11/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/lanarich/.cache/pypoetry/virtualenvs/diploma-au9TFMuk-py3.11/lib/python3.11/site-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [28]:
full_books_late_texts = [chunk_object.text for chunk_object in full_books_late]

In [ ]:
import torch
torch.cuda.empty_cache()

---

## БД с разными эмбеддинговыми моделями

In [ ]:
from langchain_core.documents import Document

documents_token = [Document(page_content=text) for text in full_books_token]
documents_rec = [Document(page_content=text) for text in full_books_rec]
documents_late = [Document(page_content=text) for text in full_books_late_texts]

In [53]:
from langchain_qdrant import Qdrant

vectorstore_token = Qdrant.from_documents(
    documents=documents_token,
    embedding=emb_model_BERTA,
    collection_name="TokenChunker_NAIVE",
    url="http://localhost:6333",
)

retriever_token = vectorstore_token.as_retriever(search_kwargs={"k": 5}) 

# vectorstore_rec = Qdrant.from_documents(
#     documents=documents_rec,
#     embedding=emb_model_BERTA,
#     collection_name="RecursiveChunker_NAIVE",
#     url="http://localhost:6333",
# )

# retriever_rec = vectorstore_rec.as_retriever(search_kwargs={"k": 5}) 

# vectorstore_rec = Qdrant.from_documents(
#     documents=documents_late,
#     embedding=emb_model_BERTA,
#     collection_name="LateChunker_NAIVE",
#     url="http://localhost:6333",
# )

# retriever_late = vectorstore_rec.as_retriever(search_kwargs={"k": 5}) 

/tmp/ipykernel_293245/3470624304.py:3: RuntimeWarning: coroutine 'VectorStore.afrom_documents' was never awaited
  vectorstore_token = Qdrant.from_documents(


In [58]:
retriever_token.invoke("123")

[Document(metadata={'_id': '5475c3b6-17ea-4ab9-a3d5-ad1235284fa2', '_collection_name': 'TokenChunker_NAIVE'}, page_content=" S 2 2 cd S ч ~ X Л>>- I >, * ё > < ё   | >< « 2 X сг 2 со 05 Л § с                                                    |                                                              |\n|------------------------------------|------------------------------------------------------------------|-------------------------------------------------------------------|-----------------------------------------------------------|------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------|--------------------------------------------------------------|\n| Через отверстия проходят 2 s а> со | cd н X X сх X ю cd «! = 2 X D 0 0                                | * О н о о, с « S м о <D гг х н cd -& 

In [54]:
import re

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(f"[Документ {i+1}]: {doc.page_content}" for i, doc in enumerate(docs))

def clean_qwen3_output(text_response: str) -> str:

    pattern = r'^\s*<think>.*?</think>\s*'
    cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)
    return cleaned_response    
  
prompt = ChatPromptTemplate.from_template("""    
Ты — экспертный помощник. Ответь на вопрос, используя только предоставленный контекст. Если ответа нет в контексте, скажи "Не знаю".
{context}   
    
Вопрос: {question} /no_think  
""")    

In [ ]:
rag_chain = (    
    {"context": retriever_token | format_docs, "question": RunnablePassthrough()}    
    # | prompt    
    # | Qwen3_8   
    # | StrOutputParser()
    # | clean_qwen3_output     
) 

# rag_chain = (    
#     {"context": retriever_rec | format_docs, "question": RunnablePassthrough()}    
#     | prompt    
#     | Qwen3_8   
#     | StrOutputParser()
#     | clean_qwen3_output     
# ) 

# rag_chain = (    
#     {"context": retriever_late | format_docs, "question": RunnablePassthrough()}    
#     | prompt    
#     | Qwen3_8   
#     | StrOutputParser()
#     | clean_qwen3_output     
# ) 

In [ ]:
response = rag_chain.invoke("Когда и какие добавочные центры окостенения образуются в эпифизах трубчатых костей, и как они влияют на формирование отростков, бугров и гребней?")    
print(response)